In [1]:
import pandas as pd
import requests
import json
import csv
import time
import datetime
from datetime import date

In [2]:
# date
after_start_date = date.fromisoformat('2016-04-01')
before_end_date = date.fromisoformat('2019-04-30')
after_start_seconds = (after_start_date -  date.fromisoformat('1970-01-01')).total_seconds()
before_end_seconds = (before_end_date -  date.fromisoformat('1970-01-01')).total_seconds()

print('after_start_seconds: ', after_start_seconds)
print('before_end_seconds: ', before_end_seconds)

after_start_seconds:  1459468800.0
before_end_seconds:  1556582400.0


# Retrieve the submissions

In [3]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
#     print(r)
    data = json.loads(r.text)
    return data['data']

def getPushshiftData_noSubreddit(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
    print(r)
    data = json.loads(r.text)
    return data['data']

In [4]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
#     author_fullName = subm['author_fullname']
    sub_id = subm['id']
    score = subm['score']
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = "None"
        
    try:
        self_text = subm['selftext']
    except KeyError:
        self_text = "None"
        
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id, title, self_text, url, author, score,subreddit, created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [5]:
# keywords list
# keywords = ['vape', 'e liquid', 'e juice', 'ejuice', 'electronic cigarette', 'eliquid', 'e cig', 'ecig', 'e hookah', 'electronic cig', 'ehookah']
keywords = ['New Jersey']

### Search keyword without specific subreddit

In [11]:
# query = "electronic-cigarette"
# sub = "electronic_cigarette"

for query in keywords:
    print('--------------------------------------')
    print('Query: ', query)
    after = str(after_start_seconds).split('.')[0]
    before = str(before_end_seconds).split('.')[0]

    subStats = {}
    subreddit_dict = {}

    data = getPushshiftData_noSubreddit(query, after, before)
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
#             print(submission)
#             print(submission['title'])
#             print(submission['subreddit'])
#             print('-----------')

            collectSubData(submission)
    #         print(submission)
            try:
                subreddit = submission['subreddit']
            except KeyError:
                subreddit = "None"
            subreddit_dict[subreddit] = subreddit_dict.get(subreddit, 0) + 1
    #         break

        # Prepare for next iteration
        print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        data = getPushshiftData_noSubreddit(query, after, before)

    print(len(data))
    print('length of subStats.keys(): ', len(subStats.keys()))
#     print('subreddit_dict: ', subreddit_dict)
    print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))
    break
#     print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))


--------------------------------------
Query:  New Jersey
<Response [200]>
1000
2016-04-20 13:45:08
<Response [200]>
1000
2016-05-13 17:46:38
<Response [200]>
1000
2016-06-01 03:39:38
<Response [200]>
1000
2016-06-17 03:19:27
<Response [200]>
1000
2016-07-06 02:45:35
<Response [200]>
1000
2016-07-25 19:02:24
<Response [200]>
1000
2016-08-12 04:30:11
<Response [200]>
1000
2016-09-03 00:53:35


KeyboardInterrupt: 

### Specific subreddit

In [7]:
# query = ""
# sub = "electronic_cigarette"
# after = str(after_start_seconds).split('.')[0]
# before = str(before_end_seconds).split('.')[0]

# data = getPushshiftData(query, after, before, sub)
# # Will run until all posts have been gathered 
# # from the 'after' date up until before date
# while len(data) > 0:
#     for submission in data:      
# #         collectSubData(submission)
#         print(submission)
# #         subCount+=1
#         break
#     # Calls getPushshiftData() with the created date of the last submission
#     print(len(data))
#     print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
#     after = data[-1]['created_utc']
#     data = getPushshiftData(query, after, before, sub)
#     break
    
# print(len(data))

In [8]:
query = ""
sub = "electronic_cigarette"
after = str(after_start_seconds).split('.')[0]
before = str(before_end_seconds).split('.')[0]


subCount = 0
subStats = {}

data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:      
        collectSubData(submission)
#         print(submission)
#         subCount+=1
#         break
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
#     break
    
print(len(data))

1000
2016-04-05 04:06:59
1000
2016-04-08 23:38:06
1000
2016-04-13 07:23:50
1000
2016-04-17 16:44:39
1000
2016-04-21 18:52:29
1000
2016-04-26 12:16:57


KeyboardInterrupt: 

In [ ]:
print('length of subStats.keys(): ', len(subStats.keys()))

In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

In [ ]:
def updateSubs_file(fileName, subStats):
    upload_count = 0
    location = "/mnt/volume-second-5T/subreddits/"
#     print("input filename of submission file, please add .csv")
#     filename = input()
    fileName = fileName + ".csv"
    file = location + fileName
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post_ID","Title","Self_text", "Url","Author","Score","Subreddit","Publish_date","Number_of_comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
        
updateSubs_file("electronic_cigarette", subStats)